In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# Extracting features of 2021 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,5.0,Hacksaw,Leone Films / Midnight Releasing,"Anthony Leone (director/screenplay); Amy Cay, ...",[3],NaN
2,JANUARY,12.0,Dr. Bird's Advice for Sad Poets,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Zu...,[4],NaN
3,JANUARY,13.0,The White Tiger,Netflix / ARRAY / Purple Pebble Pictures,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
4,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[5],NaN
...,...,...,...,...,...,...,...
360,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[277]
361,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[278]
362,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[279]
363,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[280]


In [10]:
df_2021 = df[['Title','Cast and crew']]

In [11]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,Hacksaw,"Anthony Leone (director/screenplay); Amy Cay, ..."
2,Dr. Bird's Advice for Sad Poets,Yaniv Raz (director/screenplay); Lucas Jade Zu...
3,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
4,Locked Down,Doug Liman (director); Steven Knight (screenpl...
...,...,...
360,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...
361,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...
362,American Underdog,"Erwin brothers (directors); Jon Erwin, David A..."
363,Memoria,Apichatpong Weerasethakul (director/acreenplay...


In [12]:
!pip install tmdbv3api

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from tmdbv3api import TMDb
import json
import requests
import numpy as np
tmdb = TMDb()
tmdb.api_key = 'caa7134e8101f1c78864a4713b8edf73'

In [54]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [63]:
from tmdbv3api import TMDb, Movie
import json
import requests
import numpy as np

tmdb = TMDb()
tmdb.api_key = 'caa7134e8101f1c78864a4713b8edf73'

tmdb_movie = Movie()

def get_genre(x):
    try:
        result = tmdb_movie.search(x)
        print("Search Result:", result)  # Debugging print
        
        # If the search result is empty, return NaN
        if not result:
            return np.NaN
        
        # Check if the first result has an 'id' attribute
        if hasattr(result[0], 'id'):
            movie_id = result[0].id
        else:
            return np.NaN
        
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        data_json = response.json()
        
        # Check if 'genres' key exists and if it contains any genres
        if 'genres' in data_json and data_json['genres']:
            genres = [genre['name'] for genre in data_json['genres']]
            return " ".join(genres)
        else:
            return np.NaN
    except Exception as e:
        print(f"An error occurred: {e}")
        return np.NaN

# Example usage
print(get_genre("Inception"))  # Should return the genres for Inception
print(get_genre("Nonexistent Movie"))  # Should handle gracefully and return NaN



Search Result: {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/28kKbSUvUz6P5RE1AuMJMO7IMfK.jpg', 'genre_ids': [28, 878, 12], 'id': 27205, 'original_language': 'en', 'original_title': 'Inception', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'popularity': 226.139, 'poster_path': '/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg', 'release_date': '2010-07-15', 'title': 'Inception', 'video': False, 'vote_average': 8.368, 'vote_count': 35882}, {'adult': False, 'backdrop_path': '/JeGkRdNsOuMrgwBdtB0hp763MU.jpg', 'genre_ids': [18, 53], 'id': 613092, 'original_language': 'es', 'original_title': 'El crack cero', 'overview': 'Madrid, Spain, 1975; shortly after the end of the Franco dictatorship. Six months after the mysterious death of his lover, a p

In [55]:
print(df_2021.columns)


Index(['movie_title', 'Cast and crew', 'genres', 'director_name',
       'actor_1_name', 'actor_2_name'],
      dtype='object')


In [57]:
df_2021.rename(columns={'movie_title': 'Title'}, inplace=True)

In [58]:

df_2021.loc[:, 'genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


TypeError: getattr(): attribute name must be string

In [18]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,NaN
1,Hacksaw,"Anthony Leone (director/screenplay); Amy Cay, ...",NaN
2,Dr. Bird's Advice for Sad Poets,Yaniv Raz (director/screenplay); Lucas Jade Zu...,NaN
3,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN
4,Locked Down,Doug Liman (director); Steven Knight (screenpl...,NaN
...,...,...,...
360,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,NaN
361,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,NaN
362,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",NaN
363,Memoria,Apichatpong Weerasethakul (director/acreenplay...,NaN


In [19]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_5972\3118254134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [21]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [22]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_5972\1611661901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [23]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_5972\3001516348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [25]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [26]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [43]:
df_2021

,movie_title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,NaN,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,Hacksaw,"Anthony Leone (director/screenplay); Amy Cay, ...",NaN,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess
2,Dr. Bird's Advice for Sad Poets,Yaniv Raz (director/screenplay); Lucas Jade Zu...,NaN,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes
3,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
4,Locked Down,Doug Liman (director); Steven Knight (screenpl...,NaN,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
...,...,...,...,...,...,...,...
360,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,NaN,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel
361,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,NaN,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian
362,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",NaN,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid
363,Memoria,Apichatpong Weerasethakul (director/acreenplay...,NaN,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar


In [28]:
#df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [44]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [45]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,NaN,Shadow in the Cloud
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,NaN,Hacksaw
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,NaN,Dr. Bird's Advice for Sad Poets
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,NaN,The White Tiger
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,NaN,Locked Down
...,...,...,...,...,...,...
360,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,NaN,The Tragedy of Macbeth
361,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,NaN,A Journal for Jordan
362,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,NaN,American Underdog
363,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,NaN,Memoria


In [31]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [32]:
new_df21.isna().sum()

director_name      0
actor_1_name       0
actor_2_name       8
actor_3_name      26
genres           365
movie_title        1
comb             365
dtype: int64

In [33]:
new_df21 = new_df21.dropna(how='any')

In [34]:
new_df21.isna().sum()

director_name    0.0
actor_1_name     0.0
actor_2_name     0.0
actor_3_name     0.0
genres           0.0
movie_title      0.0
comb             0.0
dtype: float64

In [35]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

In [46]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,NaN,Shadow in the Cloud
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,NaN,Hacksaw
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,NaN,Dr. Bird's Advice for Sad Poets
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,NaN,The White Tiger
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,NaN,Locked Down
...,...,...,...,...,...,...
360,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,NaN,The Tragedy of Macbeth
361,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,NaN,A Journal for Jordan
362,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,NaN,American Underdog
363,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,NaN,Memoria


In [37]:
old_df = pd.read_csv('final_data.csv')

In [47]:
print(old_df.columns)

Index(['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name',
       'genres', 'movie_title', 'comb'],
      dtype='object')


In [50]:
old_df = old_df.iloc[:, :-1]

In [51]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...
5856,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Comedy Romance,little women
5857,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Action Drama Thriller,1917
5858,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Biography Drama,just mercy
5859,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency


In [52]:
final_df = pd.concat([old_df, new_df21], ignore_index=True)

In [53]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...
6221,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,NaN,The Tragedy of Macbeth
6222,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,NaN,A Journal for Jordan
6223,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,NaN,American Underdog
6224,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,NaN,Memoria


In [41]:
final_df.to_csv('new_main_data.csv',index=False)